# Error estimation and mesh adaptivity for the Poisson equation

In this notebook, we will solve the Poisson equation (with a quite localized right hand side), estimate the error element-by-element,  refine a fraction of the elements, and continue the SOLVE - ESTIMATE - MARK - REFINE loop until a certain residual threshold is reached.

## Equation and problem definition

For a domain $\Omega \subset \mathbb{R}^n$ with boundary $\partial
\Omega = \Gamma_{D} \cup \Gamma_{N}$, the Poisson equation with
particular boundary conditions reads:

$$
\begin{align}
  - \nabla^{2} u &= f \quad {\rm in} \ \Omega, \\
  u &= 0 \quad {\rm on} \ \partial\Omega \\
\end{align}
$$

where $f$ is an input datum. The variational problem reads: find $u \in V$ such
that

$$
a(u, v) = L(v) \quad \forall \ v \in V,
$$

where $V$ is a suitable function space and

$$
\begin{align}
  a(u, v) &:= \int_{\Omega} \nabla u \cdot \nabla v \, {\rm d} x, \\
  L(v)    &:= \int_{\Omega} f v \, {\rm d} x.
\end{align}
$$

The expression $a(u, v)$ is the bilinear form and $L(v)$
is the linear form. It is assumed that all functions in $V$
satisfy the Dirichlet boundary conditions ($u = 0 \ {\rm on} \
\Gamma_{D}$).

In this demo we consider:

- $\Omega = [0,2] \times [0,1]$ (a rectangle)
- $f = 40\exp(-((x - 0.75)^2 + (y - 0.75)^2) / 0.01)$

We will solve the problem, then estimate the residuals, element by element

## Local error estimates
We will use the local, residual based error estimator given by
$$
\eta_T^{res}(u_h, f)^2 := h_T^2 \| f + \Delta u_h \|_{L_2(T)}^2 +
 \sum_{E : E \subset \partial T \atop E \subset \Omega} h_E \left\| \left[ \frac{\partial u_h}{\partial n} \right] \right\|_{L_2(E)} ^2.
$$
Here, $T$ is a mesh element, and $E$ are therefore the edges of this element, with diameter $h_T$ and length $h_E$, respectively. For a certain $u_h$ that we have compute, we can then compute, for all mesh elements $T\in\mathcal{T}$, $\eta_T^{res}(u_h, f)^2$. We then refine a fraction of the elements, where the error indicator is bigger. 
## Implementation


We import the same packages as in the Poisson example

In [1]:
from mpi4py import MPI
from petsc4py.PETSc import ScalarType  # type: ignore

In [3]:
import numpy as np

import ufl
from dolfinx import fem, io, mesh, plot, cpp
from dolfinx.fem.petsc import LinearProblem
from ufl import ds, dx, dS, grad, div, inner, avg, jump
import pyvista
pyvista.set_jupyter_backend('html')

# To remove interactivity in the plots, use
# pyvista.set_jupyter_backend('static')

The function {py:func}`solve_poisson_homog_dirichlet` solves the Poisson problem with homogeneous boundary conditions, in the same way as in the previous example

In [7]:
def solve_poisson_homog_dirichlet(msh, f, bc_marker):
    V = fem.functionspace(msh, ("Lagrange", 1))
   
    
    facets = mesh.locate_entities_boundary(
        msh,
        dim=(msh.topology.dim - 1),
        marker=bc_marker
    )
    dofs = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets)
    bc = fem.dirichletbc(value=ScalarType(0), dofs=dofs, V=V)
    u = ufl.TrialFunction(V)
    v = ufl.TestFunction(V)

    a = inner(grad(u), grad(v)) * dx
    L = inner(f, v) * dx
    problem = LinearProblem(a, L, bcs=[bc], 
                             petsc_options={"ksp_type": "preonly", "pc_type": "lu"},
                             petsc_options_prefix="poisson_adapt")
    
    
    uh = problem.solve()

    return uh, V

In [18]:
def viz(V, uh):
    cells, types, x = plot.vtk_mesh(V)
    grid = pyvista.UnstructuredGrid(cells, types, x)
    grid.point_data["u"] = uh.x.array.real
    grid.set_active_scalars("u")
    plotter = pyvista.Plotter(off_screen=True)

    warped = grid.warp_by_scalar()
    plotter.add_mesh(warped, show_edges=True)
    plotter.screenshot("uh_poisson.png")

    if not pyvista.OFF_SCREEN:
        plotter.show()

def write_to_file(msh, uh, idx):
    with io.VTKFile(msh.comm, f"out_poisson/poisson", "w") as file:
        file.write_function(uh, float(idx))

We create the mesh and the right hand side, then solve the Poisson problem the first time

In [19]:
msh = mesh.create_rectangle(
    comm=MPI.COMM_WORLD,
    points=((0.0, 0.0), (2.0, 1.0)),
    n=(4, 2),
    cell_type=mesh.CellType.triangle,
)
bc_marker = lambda x: np.isclose(x[0], 0.0) | np.isclose(x[0], 2.0) | np.isclose(x[1], 0.0) | np.isclose(x[1], 1.0)
x = ufl.SpatialCoordinate(msh)
f = 40 * ufl.exp(-((x[0] - 0.75) ** 2 + (x[1] - 0.75) ** 2) / 0.01)


uh, V = solve_poisson_homog_dirichlet(msh, f, bc_marker)

In [20]:
viz(V, uh)

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

We now create a space of piecewise constant functions: "DG" stands for discontinuous Galerkin, $0$ is the degree of the polynomials in each mesh cell (hence, we use piecewise constants).

We extract the number of mesh cells, and decide that we refine approximately $1/20$ of the cells.

In [21]:
DG = fem.functionspace(msh, ("DG", 0))
tdim = msh.topology.dim
num_cells = msh.topology.index_map(tdim).size_local
num_cells_to_refine = int(np.ceil(0.05*num_cells))

First, we get the vectors that are normal to each edge of the mesh. Then, we want to use the linear form
$$
\int_T 1 dx
$$
to compute the area of each element. We exploit the fact that the basis for the DG space is given by
$$
\{\mathbb{1}_T\}_{T\in \mathcal{T}},
$$
i.e., each basis function is $1$ on an element and zero elsewhere, to extract the vector with the areas of the elements. We then create a piecewise constant function `cell_area` that is equal, in each element, to the area of that element.

In [22]:
n = ufl.FacetNormal(msh)
w = ufl.TestFunction(DG)
cell_area_form = fem.form(w*dx)
cell_area = fem.function.Function(DG, fem.assemble_vector(cell_area_form))

We define the linear form that implements the local residual computation, using the same idea as above.

In [23]:
residual = fem.form(2*cell_area*w*(div(grad(uh))-f)**2*dx + np.sqrt(2)*avg(cell_area**(1./2))*avg(w)*jump(grad(uh),n)**2*dS)    

We now extract the vector with the local residuals and compute the global residual. We then order the cell by residual from the biggest to the smallest, store their indeces, and keep the first `num_cells_to_refine`.

In [24]:

array_cell_res = fem.assemble_vector(residual).array
tot_residual = np.sum(array_cell_res)
cell_ord_by_res = [idx for idx, value in sorted(enumerate(array_cell_res), key=lambda x: x[-1], reverse=True)]

marked_cells = cell_ord_by_res[0:num_cells_to_refine] 


We extract the "element to edge" connectivity of the mesh, then, for each cell that we have marked for refinement, extract the edges that belong to its boundary. We update the mesh by refining the cells that we have marked.

In [25]:

msh.topology.create_connectivity(msh.topology.dim,1)
c_to_e  = msh.topology.connectivity(msh.topology.dim,1)

edges=[]
for cell in marked_cells:
    for e in c_to_e.links(cell):
        edges.append(e)

msh = mesh.refine(msh, np.array(edges))[0]
x = ufl.SpatialCoordinate(msh)
f = 40 * ufl.exp(-((x[0] - 0.75) ** 2 + (x[1] - 0.75) ** 2) / 0.01)


We solve the Poisson problem on this new mesh and visualize the updated solution

In [26]:

uh, V = solve_poisson_homog_dirichlet(msh, f, bc_marker)
viz(V, uh)

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

Finally, we do the same refinement procedure, until a certain threshold in the global residual indicator is attained.

In [27]:
tot_residual = 1
iteration = 0
writer = io.VTKFile(msh.comm, f"out_poisson/poisson", "w")

while tot_residual > 5e-2:
    DG = fem.functionspace(msh, ("DG", 0))
    tdim = msh.topology.dim
    num_cells = msh.topology.index_map(tdim).size_local
    
    
    n = ufl.FacetNormal(msh)
    w = ufl.TestFunction(DG)
    cell_area_form = fem.form(w*dx)
    cell_area = fem.function.Function(DG, fem.assemble_vector(cell_area_form))
    
    residual = fem.form(2*cell_area*w*(div(grad(uh))+f)**2*dx + np.sqrt(2)*avg(cell_area**(1./2))*avg(w)*jump(grad(uh),n)**2*dS)    
     
    array_cell_res = fem.assemble_vector(residual).array
    tot_residual = np.sum(array_cell_res)
    cell_ord_by_res = [idx for idx, value in sorted(enumerate(array_cell_res), key=lambda x: x[-1], reverse=True)]
    num_cells_to_refine = int(np.ceil(0.05*num_cells))
    marked_cells = cell_ord_by_res[0:num_cells_to_refine] 
    
    msh.topology.create_connectivity(msh.topology.dim,1)
    c_to_e  = msh.topology.connectivity(msh.topology.dim,1)
    
    edges=[]
    for cell in marked_cells:
        for e in c_to_e.links(cell):
            edges.append(e)
    
    msh = mesh.refine(msh, np.array(edges))[0]
    x = ufl.SpatialCoordinate(msh)
    f = 40 * ufl.exp(-((x[0] - 0.75) ** 2 + (x[1] - 0.75) ** 2) / 0.01)
    uh, V = solve_poisson_homog_dirichlet(msh, f, bc_marker)
    iteration = iteration+1
    print(f"Iteration: {iteration}, residual: {tot_residual}")

    writer.write_function(uh, iteration)
    if iteration%3==0:
        viz(V, uh)


viz(V, uh)

Iteration: 1, residual: 3.3264738140512677
Iteration: 2, residual: 1.612465554789951
Iteration: 3, residual: 0.7209453896141136


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

Iteration: 4, residual: 0.46001014876798685
Iteration: 5, residual: 0.3398923841965564
Iteration: 6, residual: 0.25490294679510916


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

Iteration: 7, residual: 0.20122188669005944
Iteration: 8, residual: 0.15880507379466557
Iteration: 9, residual: 0.12317082488989826


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

Iteration: 10, residual: 0.09816702281721443
Iteration: 11, residual: 0.07563548065454029
Iteration: 12, residual: 0.05997847918666413


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

Iteration: 13, residual: 0.04616991660906686


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…